In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping

2023-07-01 08:59:57.741979: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-01 09:00:05.974139: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-01 09:00:05.999744: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-01 09:00:12.955221: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
csv = pd.read_csv(
    "/home/brijesh/Documents/ai/spam-detection/datasets/yt.csv",
    delimiter=","
)
df = csv[["Comment", "Spam"]]

df = df.dropna()


In [3]:

X = df['Comment'].values
y = df['Spam'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    
print(len(X_train))
print(len(X_test))

model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

print(model.summary())

11792
2948


2023-07-01 09:01:04.593138: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-01 09:01:04.594930: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-07-01 09:01:06.093066: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'sen

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14740 entries, 0 to 14739
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Comment  14740 non-null  object
 1   Spam     14740 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 230.4+ KB


In [4]:


early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# fit the model
model.fit(X_train,
         y_train,
         epochs=40,
         batch_size=512,
         validation_data=(X_test, y_test),
         verbose=1,
         callbacks=[early_stop]
)


model.save("trained-model.h5")

Epoch 1/40


2023-07-01 09:01:07.138810: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall' with dtype float and shape [?,50]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall}}]]
2023-07-01 09:01:07.138903: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_3' with dtype int64 and shape [?]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_3}}]]


24/24 [==============================] - 8s 275ms/step - loss: 0.6188 - accuracy: 0.7508 - val_loss: 0.4911 - val_accuracy: 0.9488
Epoch 2/40
24/24 [==============================] - 6s 261ms/step - loss: 0.4129 - accuracy: 0.9555 - val_loss: 0.3265 - val_accuracy: 0.9705
Epoch 3/40
24/24 [==============================] - 6s 259ms/step - loss: 0.2844 - accuracy: 0.9697 - val_loss: 0.2277 - val_accuracy: 0.9766
Epoch 4/40
24/24 [==============================] - 6s 260ms/step - loss: 0.2113 - accuracy: 0.9738 - val_loss: 0.1741 - val_accuracy: 0.9776
Epoch 5/40
24/24 [==============================] - 6s 259ms/step - loss: 0.1744 - accuracy: 0.9759 - val_loss: 0.1459 - val_accuracy: 0.9780
Epoch 6/40
24/24 [==============================] - 6s 256ms/step - loss: 0.1554 - accuracy: 0.9766 - val_loss: 0.1311 - val_accuracy: 0.9786
Epoch 7/40
24/24 [==============================] - 6s 260ms/step - loss: 0.1441 - accuracy: 0.9765 - val_loss: 0.1219 - val_accuracy: 0.9786
Epoch 8/40
24/24 

In [5]:
# model_prod = tf.keras.models.load_model('./trained-model.h5', custom_objects={'KerasLayer':hub.KerasLayer})


In [6]:
for test in ["Congratulations! You have been selected to receive a free gift worth $1000. Claim your prize by clicking this link and entering your personal information. Hurry, this offer is only available for a limited time!","good video"]:
    out = model.predict([test])
    print(out)

1/1 [==============================] - 0s 267ms/step
[[0.01069523]]
1/1 [==============================] - 0s 72ms/step
[[5.824592e-05]]
